# LUMPI Point Cloud Tutorial
This Jupyter notebook explains the point cloud example in more detail. This tutorial uses:
- the LUMPI parser to load the labels and point cloud
- the PointCloudFilter to color the background using the provided background segmentation
- the PointCloudVisualizer to render the point clouds and bounding boxes in 3D

First, the path to the SDK objects is appended and all necessary functions are imported:

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from objects.LUMPIParser import LUMPIParser
from objects.PointCloudFilter import PointCloudFilter
from objects.PointCloudVisualizer import PointCloudVisualizer
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Then the path to the LUMP dataset is defined. The following structure is expected to load all meta information and point cloud files by the **measurement_id**:
- Root
  - Measurement**1**
    - lidar
        - 0000000.ply
  - Measurement**2**
    - lidar
        - 0000000.ply

In [3]:
lp = LUMPIParser("/home/busch/LUMPI_test_data")
measurement_id = 4
lp.read_point_cloud_file_list(measurement_id)

Next, all labels are loaded for the measurements by an arbitrary path. This line expects the labels as a SGT.csv file within each measurement directory:
- Root
  - Measurement**1**
    - lidar
    - SGT.csv

In [5]:
lp.read_track(os.path.join(lp.path, "Measurement" + str(measurement_id), "Label.csv"))

reading tracks


100%|██████████| 3541/3541 [00:00<00:00, 72199.58it/s]


Next, the point cloud filter subtraction is initialized. The path to the background data can be set arbitrarily. This line expects the background data structured as follows:
- Root
  - Measurement**1**
    - background
      - x.npy
      - meta_background.txt
    - lidar
    - SGT.csv

The meta_background.txt file is used to load angle and distance resolution for each sensor.

In [6]:
filter = PointCloudFilter()
filter.read_background(os.path.join(lp.path, "Measurement" + str(measurement_id), "background"))

Next, the viewer is initialized by the first point cloud and waits for adapting the perspective.

In [7]:
lp.read_point_cloud(0)
vis = PointCloudVisualizer()
vis.init_camera(lp.get_xyz(), 100)

Finally, all point clouds are iterated through by performing the following steps:
- clear view
- load point cloud
- get foreground and background point indices
- color foreground black and background gray
- add colored cloud to view
- iterate through all labels at point cloud index
  - add bounding box to viewer
- render the viewer and update the perspective

In [8]:
for i in range(len(lp.point_cloud_files)):
    vis.vis.clear_geometries()
    lp.read_point_cloud(i)
    # Filter background
    f, b = filter.filter_background(lp.get_points_meta())
    # Color background and foreground
    colors = np.zeros((lp.get_xyz().shape[0], 3))
    colors[f] = [0, 0, 0]
    colors[b] = [0.7, 0.7, 0.7]
    vis.add_colored_cloud(lp.get_xyz(), colors)
    # Plot bounding boxes
    if i in lp.indexOrdered:
        for o in lp.indexOrdered[i].values():
            vis.add_bounding_box(o)
    vis.update_view()

KeyboardInterrupt: 